In [1]:
!mkdir -p /data/coco/

# Download the annotation ZIP and extract it, then remove the ZIP file
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -q annotations_trainval2017.zip -d /data/coco/
!rm annotations_trainval2017.zip

# Download the train2017 ZIP, extract it, and remove the ZIP file
# !wget http://images.cocodataset.org/zips/train2017.zip
# !unzip -q train2017.zip -d /data/coco/
# !rm train2017.zip

# Download the val2017 ZIP, extract it, and remove the ZIP file
!wget http://images.cocodataset.org/zips/val2017.zip
!unzip -q val2017.zip -d /data/coco/
!rm val2017.zip

--2024-08-08 06:43:24--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.205.251, 3.5.27.228, 54.231.169.9, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.205.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241.19M  93.6MB/s    in 2.6s    

2024-08-08 06:43:27 (93.6 MB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

--2024-08-08 06:43:40--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 3.5.28.251, 52.217.204.177, 52.217.102.244, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|3.5.28.251|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val201

In [1]:
!pip install matplotlib==3.5 opencv-python pybboxes

In [2]:
!git clone https://github.com/Cartucho/mAP

Cloning into 'mAP'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 908 (delta 135), reused 125 (delta 125), pack-reused 758
Receiving objects: 100% (908/908), 14.70 MiB | 28.95 MiB/s, done.
Resolving deltas: 100% (329/329), done.


In [3]:
import json
import os
from pybboxes import BoundingBox
import pybboxes as pbx

In [8]:
# def coco_to_map_gt(coco_annotation_path, output_dir):
#     with open(coco_annotation_path, 'r') as f:
#         coco_data = json.load(f)

#     annotations = coco_data['annotations']
#     images = {image['id']: image['file_name'] for image in coco_data['images']}
#     categories = {category['id']: category['name'] for category in coco_data['categories']}

#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)

#     for annotation in annotations:
#         image_id = annotation['image_id']
#         image_file = images[image_id]
#         category_id = annotation['category_id']
#         category_name = categories[category_id]
#         bbox = annotation['bbox']

#         # Convert COCO bbox (x, y, width, height) to (xmin, ymin, xmax, ymax)
#         xmin, ymin, width, height = bbox
#         xmax = xmin + width
#         ymax = ymin + height

#         gt_file = os.path.join(output_dir, image_file.replace('.jpg', '.txt'))
#         with open(gt_file, 'w') as f:
#             f.write(f"{category_name} {xmin} {ymin} {xmax} {ymax}\n")

# coco_to_map_gt('/data/coco/annotations/instances_val2017.json', 'mAP/input/ground-truth/')


In [4]:
def coco_to_map(coco_annotation_path, output_dir, mode='ground-truth'):
    with open(coco_annotation_path, 'r') as f:
        coco_data = json.load(f)

    annotations = coco_data['annotations']
    images = {image['id']: image['file_name'] for image in coco_data['images']}
    categories = {category['id']: category['name'].replace(' ', '_') for category in coco_data['categories']}

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for annotation in annotations:
        image_id = annotation['image_id']
        if image_id not in images:
            print(f"Error. Image not found with id: {image_id}")
            continue

        image_file = images[image_id]
        category_id = annotation['category_id']
        category_name = categories[category_id]
        bbox = annotation['bbox']

        # Convert COCO bbox (x, y, width, height) to (xmin, ymin, xmax, ymax)
        xmin, ymin, width, height = bbox
        xmax = xmin + width
        ymax = ymin + height

        output_file = os.path.join(output_dir, image_file.replace('.jpg', '.txt'))

        if mode == 'ground-truth':
            with open(output_file, 'w') as f:
                f.write(f"{category_name} {xmin} {ymin} {xmax} {ymax}\n")
        elif mode == 'detection-results':
            score = 1.0  # Assuming 100% accuracy for testing
            with open(output_file, 'w') as f:
                f.write(f"{category_name} {score} {xmin} {ymin} {xmax} {ymax}\n")
        else:
            raise ValueError("Mode must be 'ground-truth' or 'detection-results'")



In [5]:
# Prepare ground-truth files
coco_to_map('/data/coco/annotations/instances_val2017.json', 'mAP/input/ground-truth/', mode='ground-truth')

# Prepare detection-results files
coco_to_map('/data/coco/annotations/instances_val2017.json', 'mAP/input/detection-results/', mode='detection-results')

In [17]:
%cd /content/

/content


In [6]:
%cd /content/
%cd mAP
!python main.py -na
%cd /content/

/content
/content/mAP
100.00% = airplane AP 
/content/mAP/main.py:704: MatplotlibDeprecationWarning: 
The set_window_title function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use manager.set_window_title or GUI-specific methods instead.
  fig.canvas.set_window_title('AP ' + class_name)
100.00% = apple AP 
100.00% = backpack AP 
100.00% = banana AP 
100.00% = baseball_bat AP 
100.00% = baseball_glove AP 
100.00% = bear AP 
100.00% = bed AP 
100.00% = bench AP 
100.00% = bicycle AP 
100.00% = bird AP 
100.00% = boat AP 
100.00% = book AP 
100.00% = bottle AP 
100.00% = bowl AP 
100.00% = broccoli AP 
100.00% = bus AP 
100.00% = cake AP 
100.00% = car AP 
100.00% = carrot AP 
100.00% = cat AP 
100.00% = cell_phone AP 
100.00% = chair AP 
100.00% = clock AP 
100.00% = couch AP 
100.00% = cow AP 
100.00% = cup AP 
100.00% = dining_table AP 
100.00% = dog AP 
100.00% = donut AP 
100.00% = elephant AP 
100.00% = fire_hydrant AP 
100.00% = fork AP 
100.00% =

## All type conversation

In [4]:
def coco_to_map_with_conversion(coco_annotation_path, output_dir, mode='ground-truth', target_format='coco'):
    with open(coco_annotation_path, 'r') as f:
        coco_data = json.load(f)

    annotations = coco_data['annotations']
    images = {image['id']: image['file_name'] for image in coco_data['images']}
    image_sizes = {image['id']: (image["width"], image['height']) for image in coco_data['images']}
    categories = {category['id']: category['name'].replace(' ', '_') for category in coco_data['categories']}

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for annotation in annotations:
        image_id = annotation['image_id']
        if image_id not in images:
            print(f"Error. Image not found with id: {image_id}")
            continue

        image_file = images[image_id]
        image_size = image_sizes[image_id]

        category_id = annotation['category_id']
        category_name = categories[category_id]
        bbox = annotation['bbox']

        if target_format == 'voc':
            converted_bbox = pbx.convert_bbox(bbox, from_type="coco", to_type="voc")
        elif target_format == 'albumentations':
            converted_bbox = pbx.convert_bbox(bbox, from_type="coco", to_type="albumentations", image_size=image_size)
        elif target_format == 'yolo':
            converted_bbox = pbx.convert_bbox(bbox, from_type="coco", to_type="yolo", image_size=image_size)
        elif target_format == 'coco':
            converted_bbox = bbox
        else:
            raise ValueError("Unsupported target format")

        # Convert back to COCO format for evaluation
        if target_format != 'coco':
            converted_back_bbox = pbx.convert_bbox(converted_bbox, from_type=target_format, to_type="coco", image_size=image_size)
        else:
            converted_back_bbox = converted_bbox

        output_file = os.path.join(output_dir, image_file.replace('.jpg', '.txt'))

        xmin, ymin, width, height = converted_back_bbox
        xmax = xmin + width
        ymax = ymin + height

        if mode == 'ground-truth':
            with open(output_file, 'w') as f:
                f.write(f"{category_name} {xmin} {ymin} {xmax} {ymax}\n")
        elif mode == 'detection-results':
            score = 1.0  # Assuming 100% accuracy for testing
            with open(output_file, 'w') as f:
                f.write(f"{category_name} {score} {xmin} {ymin} {xmax} {ymax}\n")
        else:
            raise ValueError("Mode must be 'ground-truth' or 'detection-results'")


**Needs to restart result before running**

In [13]:
!rm -r /content/mAP/input/ground-truth/
!rm -r /content/mAP/input/detection-results/
!rm -r /content/mAP/input/images-optional

rm: cannot remove '/content/mAP/input/images-optional': No such file or directory


In [14]:
coco_to_map_with_conversion('/data/coco/annotations/instances_val2017.json', 'mAP/input/ground-truth/', mode='ground-truth', target_format='albumentations')

coco_to_map_with_conversion('/data/coco/annotations/instances_val2017.json', 'mAP/input/detection-results/', mode='detection-results', target_format='albumentations')

In [35]:
# Load the JSON data
with open('/data/coco/annotations/instances_val2017.json', 'r') as f:
    coco_data = json.load(f)

# Print the top-level keys
print("Top-level keys:", coco_data.keys())

# Print a sample of the images and annotations
print("Sample of images:")
for i in range(3):
    print(coco_data['images'][i])

print("\nSample of annotations:")
for i in range(3):
    print(coco_data['annotations'][i])


Top-level keys: dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])
Sample of images:
{'license': 4, 'file_name': '000000397133.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-14 17:02:52', 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg', 'id': 397133}
{'license': 1, 'file_name': '000000037777.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000037777.jpg', 'height': 230, 'width': 352, 'date_captured': '2013-11-14 20:55:31', 'flickr_url': 'http://farm9.staticflickr.com/8429/7839199426_f6d48aa585_z.jpg', 'id': 37777}
{'license': 4, 'file_name': '000000252219.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000252219.jpg', 'height': 428, 'width': 640, 'date_captured': '2013-11-14 22:32:02', 'flickr_url': 'http://farm4.staticflickr.com/3446/3232237447_13d84bd0a1_z.jpg', 'id': 252219}

Sample of annotations:
{'segmentation': [[510.66, 42

In [15]:
%cd /content/
%cd mAP
!python main.py -na
%cd /content/

/content
/content/mAP
100.00% = airplane AP 
/content/mAP/main.py:704: MatplotlibDeprecationWarning: 
The set_window_title function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use manager.set_window_title or GUI-specific methods instead.
  fig.canvas.set_window_title('AP ' + class_name)
100.00% = apple AP 
100.00% = backpack AP 
100.00% = banana AP 
100.00% = baseball_bat AP 
100.00% = baseball_glove AP 
100.00% = bear AP 
100.00% = bed AP 
100.00% = bench AP 
100.00% = bicycle AP 
100.00% = bird AP 
100.00% = boat AP 
100.00% = book AP 
100.00% = bottle AP 
100.00% = bowl AP 
100.00% = broccoli AP 
100.00% = bus AP 
100.00% = cake AP 
100.00% = car AP 
100.00% = carrot AP 
100.00% = cat AP 
100.00% = cell_phone AP 
100.00% = chair AP 
100.00% = clock AP 
100.00% = couch AP 
100.00% = cow AP 
100.00% = cup AP 
100.00% = dining_table AP 
100.00% = dog AP 
100.00% = donut AP 
100.00% = elephant AP 
100.00% = fire_hydrant AP 
100.00% = fork AP 
100.00% =